In [1]:
import pandas as pd
import numpy as np
import os
import requests
from io import BytesIO
import json
import yadisk
from datetime import datetime, date, timedelta
import re
import gspread
from oauth2client.service_account import ServiceAccountCredentials

import config
from yandex_disk_func import *
from parse_functions import *
from google_connector import *

# # указываем путь и название файла с токеном для Яндекс Диск
file_path = r'C:\Users\o.bogomolov\Desktop\Jupyter_notebook\35_mediaplan_parser'
# забираем токен для подключения к гугл
service_key = config.service
google_sheet_link = 'https://docs.google.com/spreadsheets/d/11hF_txYEbKrQImRQ0x3Fia3PyJvDAqtpGI25idV7oa4/edit?usp=sharing'
gmail = 'ads@ichance.ru'
sheet_name = 'prog'

# забираем Яндекс токен
yandex_token = config.yandex_token
# # указываем путь к основной папке, в которой храняться папки с флайтами
main_folder = config.main_folder
public_key = config.public_key # обычная ссылка на доступ к папке одного данного ФЛАЙТА из личного кабинета

# забираем список папок в нужной нам директории
res = get_yandex_disk_folders(public_key)
yandex_folders = res.json() # парсим ответ

200


In [2]:
# Включаем отображение всех колонок
pd.set_option('display.max_columns', None)
# Задаем ширину столбцов по контенту
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)
pd.options.mode.chained_assignment = None
# pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [3]:
# создаем словарь, где сохраним каждый продукт в отдельный датаФрейм со своим ключом
main_dict = {}

In [4]:
get_data_from_ya_folder(main_folder, yandex_folders, yandex_token, main_dict, flag='prog')                     


/01_yandex
/02_prog
beeline_plan_игроник_ингосстрах_кис_лето_banners_020724
firstdata_игроник_ингосстрах_срм_media_28.08.2024
mediaplan без ск
firstdata_игроник_национальная лотерея_принцесса нури_срм_media_27.08.2024 (1)
mediaplan 1000к
mediaplan 800к
mediaplan 600к
plan_игроник_рсхб_beeline_свой вклад_banners_160724 (1)
plan_ингосстрах_ beeline_хореографы_banners_240924 (3)
beeline_plan_igronik_бриф по тендерам_101024
/03_Полина


In [5]:
df = pd.concat(main_dict, ignore_index=True)
df = df.fillna('')


int_columns = ['unit quantity', 'reach', 'impressions', 'clicks']
df[int_columns] = df[int_columns].apply(lambda x: x.astype('int'))


In [6]:
float_columns = ['unit price', 'budget_without_nds', 'budget_nds', 'vtr, %']
df[float_columns] = df[float_columns].apply(lambda x: x.astype('float')).round(2)

In [16]:
test = df[df['sheet_name']=='mediaplan 600к']

In [ ]:
test.iloc[1]

In [11]:
test[['unit quantity', 'reach', 'impressions', 'clicks', 'budget_without_nds']].sum()

unit quantity         35550000.0
reach                  7110000.0
impressions           35550000.0
clicks                   71100.0
budget_without_nds     5321250.0
dtype: float64

In [ ]:
# создаем подключение к Гуглу
client = create_connection(service_key)

In [ ]:
# прописываем путь и подключаемсяк нужному листу Гугл докса
sh = client.open_by_url(google_sheet_link)
sh.share(gmail, perm_type='user', role='writer')
google_sheet = sh.worksheet(sheet_name)

In [ ]:
# выгружем наш датаФрейм на лист
google_sheet.clear()
export_dataframe_to_google_sheet(google_sheet, final_report)

In [ ]:
main_dict['beeline_plan_игроник_ингосстрах_кис_лето_banners_020724']

In [ ]:
main_dict['plan_игроник_рсхб_beeline_свой вклад_banners_160724 (1)'].iloc[0]

In [ ]:
main_dict['plan_ингосстрах_ beeline_хореографы_banners_240924 (3)']

In [ ]:
main_dict['beeline_plan_igronik_бриф по тендерам_101024']